In [1]:
import numpy as np
import json

In [ ]:
S_base = 10e6
P_inv_max = 3e6 # W
S_inv_max = P_inv_max
U_lv = 800
U_mv = 20e3
U_hv = 132e3

F = 50

M = 2
N = 3


S_bess_mva = 1
S_bess_storage_kWh = 250

S_plant = S_inv_max*N*M + 1.2*S_bess_mva*1e6

Irrad_max = 1000 # W/m_2
Area_form_factor = 1.0
PV_efficiency = 0.1

Z_trafo_poi_pu = 0.1
S_trafo_poi = M*N*S_inv_max*1.1

# Area_inv = Long_n * Long_m    
# Long_n =  Area_form_factor * Long_m
# Area_inv = Area_form_factor * Long_m**2

Area_inv = P_inv_max/(Irrad_max*PV_efficiency)
Long_m = np.sqrt( Area_inv/Area_form_factor )
Long_n =  Area_form_factor * Long_m



S_grid_line = S_plant*2.0

data = {
    "system":{"name":f"pv_{M}_{N}","S_base":S_base,"K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},
    "buses":[
        {"name":"POIMV","P_W":0.0,"Q_var":0.0,"U_kV":U_mv/1e3},
        {"name":   "POI","P_W":0.0,"Q_var":0.0,"U_kV":U_hv/1e3},
        {"name":  "GRID","P_W":0.0,"Q_var":0.0,"U_kV":U_hv/1e3},
        {"name":"BESS","P_W":0.0,"Q_var":0.0,"U_kV":U_lv/1e3},
    ],
    "lines":[
        {"bus_j":"POI","bus_k":"GRID","X_pu":0.001*S_base/S_grid_line,"R_pu":0.0,"Bs_pu":0.0,"S_mva":S_base/1e6, 'sym':True, 'monitor':True},
        {"bus_j":"BESS","bus_k": "POIMV","X_pu":0.01,"R_pu":0.0,"Bs_pu":0.0,"S_mva":S_base/1e6, 'sym':True, 'monitor':True},
        ],
    "transformers":[{"bus_j":"POIMV","bus_k": "POI","X_pu":0.05*S_base/S_grid_line,"R_pu":0.0,"Bs_pu":0.0,"S_mva":S_base/1e6}],
    "pvs":[],
    "sources":[{"type":"genape","bus":"GRID",
                "S_n":1000e6,"F_n":50.0,"X_v":0.001,"R_v":0.0,
                "K_delta":0.001,"K_alpha":1e-6}],
    "vscs":[{"type":"bess_pq","bus":"BESS","E_kWh":S_bess_storage_kWh,"S_n":S_bess_mva*1e6,
            "soc_ref":0.5,
            "socs":[0.0, 0.1, 0.2, 0.8,0.9,1.0],
            "es":[1, 1.08, 1.13, 1.17, 1.18,1.25]}
        ],
    }




In [3]:
cables = [
{'name':  '1X95', 'R_dc_km': 0.320, 'R_ac_km': 0.403, 'X_km': 0.128, 'muC_km': 0.187, 'R_0_km': 1.050, 'X_0_km': 0.391, 'muC_0_km': 0.187,  'I_max_aire': 255, 'I_max_direct': 205, 'I_max_tube': 190, 'I_cc_max':  8930, 'I_cc_0_max': 3140},
{'name': '1X150', 'R_dc_km': 0.206, 'R_ac_km': 0.262, 'X_km': 0.119, 'muC_km': 0.216, 'R_0_km': 0.890, 'X_0_km': 0.341, 'muC_0_km': 0.216,  'I_max_aire': 335, 'I_max_direct': 260, 'I_max_tube': 245, 'I_cc_max': 14100, 'I_cc_0_max': 3470},
{'name': '1X240', 'R_dc_km': 0.125, 'R_ac_km': 0.161, 'X_km': 0.109, 'muC_km': 0.260, 'R_0_km': 0.768, 'X_0_km': 0.297, 'muC_0_km': 0.260,  'I_max_aire': 455, 'I_max_direct': 345, 'I_max_tube': 320, 'I_cc_max': 22600, 'I_cc_0_max': 3810},
{'name': '1X400', 'R_dc_km':0.0778, 'R_ac_km': 0.102, 'X_km': 0.102, 'muC_km': 0.313, 'R_0_km': 0.650, 'X_0_km': 0.237, 'muC_0_km': 0.313,  'I_max_aire': 610, 'I_max_direct': 445, 'I_max_tube': 415, 'I_cc_max': 37600, 'I_cc_0_max': 4300},
{'name': '1X500', 'R_dc_km':0.0605, 'R_ac_km': 0.084, 'X_km': 0.099, 'muC_km': 0.329, 'R_0_km': 0.618, 'X_0_km': 0.225, 'muC_0_km': 0.329,  'I_max_aire': 715, 'I_max_direct': 505, 'I_max_tube': 480, 'I_cc_max': 47000, 'I_cc_0_max': 4810},
{'name': '1X630', 'R_dc_km':0.0469, 'R_ac_km':0.0636, 'X_km': 0.095, 'muC_km': 0.396, 'R_0_km': 0.561, 'X_0_km': 0.195, 'muC_0_km': 0.396,  'I_max_aire': 830, 'I_max_direct': 575, 'I_max_tube': 545, 'I_cc_max': 59200, 'I_cc_0_max': 5140}
]


Z_base_trafo_poi = U_mv**2/S_trafo_poi 
Z_trafo_poi = Z_base_trafo_poi*Z_trafo_poi_pu
I_cc_max = U_mv/(np.sqrt(3)*Z_trafo_poi)

for it, item in enumerate(cables):
    if item['I_cc_max'] > I_cc_max:
        print(f"Short circuit designed cable ({I_cc_max/1e3:0.2f} kA): {item['name']}")
        idx_cc = it
        break

I_max = 1.0*N*S_inv_max/(np.sqrt(3)*U_mv)

for it, item in enumerate(cables):
    if item['I_max_tube'] > I_max:
        print(f"Nominal current designed cable  ({I_max:0.2f} A): {item['name']}")
        idx_n = it
        break

if idx_cc > idx_n:
    idx_cable = idx_cc
else:
    idx_cable = idx_n

print(f"Cable: {cables[idx_cable]['name']}")

R_km_cable =  cables[idx_cable]['R_ac_km']
X_km_cable =  cables[idx_cable]['X_km']
B_km_cable =  2*np.pi*50*cables[idx_cable]['muC_km']*1e-6

Short circuit designed cable (5.72 kA): 1X95
Nominal current designed cable  (259.81 A): 1X240
Cable: 1X240


In [4]:
I_mp = 3.56
V_mp = 33.7
V_dc_n = 800
P_mp = I_mp*V_mp
N_pv_s = int(V_dc_n/V_mp)
N_pv_p = int(P_inv_max/(P_mp*N_pv_s))

In [6]:

S_feeder = N*S_inv_max
I_feeder = S_feeder/(np.sqrt(3)*U_mv)

pos_mv_poi_m_pu = 0.0
pos_mv_poi_m = pos_mv_poi_m_pu * (M-1)*Long_m
pos_mv_poi_n = 0.0

for i_m in range(1,M+1):
    name_j = "POIMV"
    monitor = True
    pos_m =  (i_m-1) * Long_m


    for i_n in range(1,N+1):
        name = f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
        name_k = 'MV' + name

        data['buses'].append({"name":f"LV{name}","P_W":0.0,"Q_var":0.0,"U_kV":U_lv/1e3})
        data['buses'].append({"name":f"MV{name}","P_W":0.0,"Q_var":0.0,"U_kV":U_mv/1e3})


        if i_n == 1:
            Long = np.abs(pos_mv_poi_m - pos_m) + Long_n*0.5
        else:
            Long = Long_n

        pos_n =  (i_n-1) * Long_n + Long_n/2
        print(f'pos_m = {pos_m:5.1f}, pos_n = {pos_n:5.1f}')

        # LV-MV Trafos
        S_trafo_n = 1.2*S_inv_max
        X_pu = 0.05*S_base/S_trafo_n
        R_pu = 0.01*S_base/S_trafo_n
        data['lines'].append({"bus_j":f"LV{name}","bus_k":f"MV{name}","X_pu":X_pu,"R_pu":R_pu,"Bs_pu":0.0,"S_mva":S_base/1e6,"monitor":False})
       
        # MV Cables:
        S_b = S_base
        Z_b = U_mv**2/S_b
        R_cable_pu = R_km_cable*Long/1e3/Z_b
        X_cable_pu = X_km_cable*Long/1e3/Z_b
        B_cable_pu = B_km_cable*Long/1e3*Z_b

        
        data['lines'].append({"bus_j":f"{name_k}","bus_k":f"{name_j}","X_pu":X_cable_pu,"R_pu":R_cable_pu,"Bs_pu":B_cable_pu,"S_mva":S_base/1e6,"monitor":monitor})
        name_j = name_k
        data['pvs'].append({"bus":f"LV{name}","type":"pv_dq_d","S_n":S_inv_max,"U_n":U_lv,"F_n":50.0,"X_s":0.1,"R_s":0.0001,"monitor":False,
                            "I_sc":8,"V_oc":42.1,"I_mp":I_mp,"V_mp":V_mp,"K_vt":-0.160,"K_it":0.065,"N_pv_s":N_pv_s,"N_pv_p":N_pv_p})
    
        monitor = False



with open(f'pv_{M}_{N}.json','w') as fobj:
    fobj.write(json.dumps(data, indent=2))

import os

print(os.getcwd())

pos_m =   0.0, pos_n =  86.6
pos_m =   0.0, pos_n = 259.8
pos_m =   0.0, pos_n = 433.0
pos_m = 173.2, pos_n =  86.6
pos_m = 173.2, pos_n = 259.8
pos_m = 173.2, pos_n = 433.0
c:\Users\jmmau\workspace\cocoon\copv_2_3\pv_2_3_bess


In [45]:
X_POI_pu = 0.1

S_POI_n = M*N*S_inv_max*1.1
I_POI_n = S_POI_n/(np.sqrt(3)*20e3)
I_POI_n/X_POI_pu

5715.767664977295

In [46]:
json.dumps(data, indent=2)

'{\n  "system": {\n    "name": "pv_2_3",\n    "S_base": 10000000.0,\n    "K_p_agc": 0.0,\n    "K_i_agc": 0.0,\n    "K_xif": 0.01\n  },\n  "buses": [\n    {\n      "name": "POIMV",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 20.0\n    },\n    {\n      "name": "POI",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 132.0\n    },\n    {\n      "name": "GRID",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 132.0\n    },\n    {\n      "name": "BESS",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 0.8\n    },\n    {\n      "name": "LV0101",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 0.8\n    },\n    {\n      "name": "MV0101",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 20.0\n    },\n    {\n      "name": "LV0102",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 0.8\n    },\n    {\n      "name": "MV0102",\n      "P_W": 0.0,\n      "Q_var": 0.0,\n      "U_kV": 20.0\n    },\n    {\n      "name": "LV0103",\n      "P_W": 0.0,\n      "Q